# Building a choropleth map

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import folium
from time import sleep

%load_ext autoreload
%autoreload 2

## Loading & wrangling the data 

In [2]:
data_path = 'data/P3_GrantExport.xlsx'
grant_data = pd.read_excel(data_path)

We start by selecting the features we're interested in. Obviously, we need the grant given to a project. We also want the University (from which we will retrieve the canton) and the Institution (since for some projects one is missing, but we're somehow confident we can retrieve the canton from either of those two). At last, we decide for now to save the reason a project got a funding, without being sure whether or not it will be usefull.



In [3]:
grant_data = grant_data[["Approved Amount", "University", "Institution","Funding Instrument Hierarchy"]]
grant_data.count()

Approved Amount                 63967
University                      50988
Institution                     58831
Funding Instrument Hierarchy    62915
dtype: int64

For now we did not remove any entries. A more thorough visualization gave us the confirmation that we should though. We start by removing entries for which the Approved Amount is not a number, since we won't be able to use those data in this study.

In [4]:
grant_data = grant_data[grant_data['Approved Amount'].apply(lambda x: str(x).isdigit())]
grant_data.count()

Approved Amount                 52663
University                      50487
Institution                     48205
Funding Instrument Hierarchy    51620
dtype: int64

As we can see we already removed about 11K entries, but we can do better. Indeed, we will use the Google's API to link a university name or an institution to a canton. So far we are confident that we can find the canton from either of those two. That means however, that we cannot treat data that miss both those values.

In [5]:
grant_data = grant_data.dropna(subset=["Institution", "University"], how="all")
grant_data.count()

Approved Amount                 51843
University                      50487
Institution                     48205
Funding Instrument Hierarchy    50800
dtype: int64

We removed another 1K entries. But we saw a special value in the university field: Nicht zuteilbar - NA, for which we won't be able to retrieve the canton.

In [6]:
grant_data = grant_data[grant_data["University"] != "Nicht zuteilbar - NA"]
grant_data.count()

Approved Amount                 49252
University                      47896
Institution                     48025
Funding Instrument Hierarchy    48231
dtype: int64

We observe that for many entries, an acronym is given at the end of the University field. A first logical step to do before moving to an API is to retrieve this acronym, and when possible, associate it to a canton (sometimes, it is already the canton's abbreviation itself). We will then have to  use the API on the remaining, undetermined entries.

In [7]:
grant_data["shorthand"] = grant_data["University"].apply(lambda x : str(x).split("- ")[-1])
grant_data["shorthand"].value_counts()

ZH                           6704
GE                           6346
ETHZ                         6093
BE                           5422
BS                           4685
EPFL                         4370
LA                           4062
FR                           2041
NE                           1580
NPO                          1463
nan                          1356
PSI                           535
FP                            490
SG                            424
USI                           338
EAWAG                         330
HES-SO                        270
ZFH                           256
EMPA                          236
FHNW                          221
WSL                           220
LU                            211
IHEID                         194
BFH                           136
AGS                           135
SUPSI                         134
FMI                            83
IDIAP                          81
ASPIT                          81
HSLU          

With this trick we can already sort a large portion of the dataframe. We will now create a feature "Canton" which we can fill with values of "Shorthand" we can associate for a canton (including EPFL and ETH in VD and ZH because, come on), and let blank for others.

In [8]:
#We already know the locations of some frequent occurences and it seems better to call the API on the fewer cases possible
grant_data.replace(["EPFL", "ETHZ", "LA"], ["VD", "ZH", "VD"], inplace=True)

In [9]:
# data needed by parse_name_for_cantons
abrv_path = 'data/cantons.csv'
abrv_data = pd.read_csv(abrv_path, sep=',')

# extract acronyms
acronyms = abrv_data['acronym'].copy()

# set acronyms as index for later
abrv_data.set_index('canton', inplace=True)

In [10]:
grant_data["Canton"] = grant_data["shorthand"][grant_data["shorthand"].isin(acronyms.values)]
grant_data["Canton"].value_counts()
#grant_data["Canton"].describe()

ZH    12797
VD     8432
GE     6346
BE     5422
BS     4685
FR     2041
NE     1580
SG      424
LU      211
Name: Canton, dtype: int64

That's 40K less google searches ! :D

## Dealing with non obvious values

Despite the fact that we have considerably reduced the number of unknown Cantons, we still try to investigate about 7000 university names. The limit of requests for the Google Maps Geocoding API is requests 2'500 per day so we need an other approach to solve this problem. Luckily, there are a lot of duplicates in the university name and they don't needed to be investigated twice.

In [11]:
# remove duplicates and missing
unknown_uni = grant_data[pd.isnull(grant_data['Canton'])]['University'].dropna().drop_duplicates().copy().to_frame()

# checking that there are not too many entry left
print(unknown_uni.count())

University    65
dtype: int64


To parse the name in the University column, we define the function find_canton_abrv. This function takes a string with some clue about the location and then try to find the location from this clue using the geocoder library and Google Maps Geocoding API. Finally, we use the find the canton name and convert it to an abreviation using the file cantons.csv.

In [12]:
# library used to parse location
import geocoder

def find_canton_abrv(clue):
    """Find canton abreviation. 
    Take a string containg clues as input and
    output a string of the abreviation or 
    1) NaN if not found
    2) - if not in Switzerland"""
    
    # find location from clue
    g = geocoder.google(clue)
    sleep(1)
    
    if g.country == 'CH':
        return g.state
    
    elif g.country == None:
        return np.nan
    else:
        return '-'

We can now try to fetch information on the unknown university (cell takes about 1 minute to execute)

In [13]:
# use find_canton_abrv
unknown_uni['Canton'] = unknown_uni['University'].apply(lambda x : find_canton_abrv(str(x).split(" -")[0]))

In [14]:
print(unknown_uni['Canton'].isnull().value_counts())
unknown_uni.head(10)

True     46
False    19
Name: Canton, dtype: int64


,University,Canton
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",NaN
29,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",NaN
56,Inst. de Hautes Etudes Internat. et du Dév - I...,NaN
83,Weitere Institute - FINST,NaN
89,Firmen/Privatwirtschaft - FP,NaN
138,Pädagogische Hochschule Graubünden - PHGR,NaN
207,Pädagogische Hochschule Zürich - PHZFH,ZH
674,Schweiz. Institut für Kunstwissenschaft - SIK-...,NaN
704,SUP della Svizzera italiana - SUPSI,NaN
748,HES de Suisse occidentale - HES-SO,NaN


In [15]:
unknown_uni.to_csv("data/still_unknown_uni.csv")

We process the unknown university manually and load them

In [16]:
processed_unknown_uni_path = 'data/processed_unknown_uni.csv'
processed_unknown_uni = pd.read_csv(processed_unknown_uni_path, sep=',')

In [17]:
for row in processed_unknown_uni.iterrows():
    grant_data.loc[grant_data['University'].isin([row[1]['University']]), 'Canton'] = row[1]['Canton']

We can save cleaned data for later use without needing to run again the slow cell above

In [18]:
grant_data.to_csv("data/data.csv")

We should finally be able to retrieve the canton feature in this dataframe.

## Creating the map

We have to manipulate a little bit the data frame first. It makes a bit more sense to use the means instead of the sums to check differences. Otherwise we would simply see a huge difference between ZH (ETHZ) and VD (EPFL) with the others cantons, since they hold so many more projects.

In [19]:
canonical_grant_data = grant_data[["Approved Amount", "Canton"]]
canonical_grant_data = canonical_grant_data.groupby(["Canton"], as_index=False).mean()
canonical_grant_data

,Canton,Approved Amount
0,-,233551
1,AG,221736
2,BE,275470
3,BS,287745
4,FR,220111
5,GE,285328
6,GR,207951
7,JU,125782
8,LU,183649
9,NE,248728


We already want to create a scale for the plot:

In [20]:
scale = np.linspace(min(canonical_grant_data["Approved Amount"]), max(canonical_grant_data["Approved Amount"]), 5).tolist()
scale

[70816.0, 125048.25, 179280.5, 233512.75, 287745.0]

However for Folium to work we need to have a match between the indices of the DF with the abbreviations present in the json file. Thus we add null values to the DF

In [21]:
for x in acronyms.values:
    if x not in canonical_grant_data["Canton"].values:
        canonical_grant_data = canonical_grant_data.append({"Canton" : x, "Approved Amount" : 0}, ignore_index=True)

In [22]:
geo_str = 'ch-cantons.topojson.json'
ch_map = folium.Map(location=[46.6430788,8.018626], tiles='Mapbox Bright', zoom_start=7)
ch_map.choropleth(geo_path=geo_str, 
                     data=canonical_grant_data,
                     columns=["Canton", "Approved Amount"],
                     threshold_scale=scale,
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,   
                     legend_name = 'Average grant data per project'
                    )
ch_map.save('chmap.html')


In [23]:
%%HTML
<iframe width='100%' height="350" src="chmap.html"></iframe>

## Conclusion

It appears that on average, Geneva gets more money than the other cantons for a research project. This could potentially be linked to the international interest the city gets, and the involvement of many multinational firms in said projects. 